In [ ]:
def createPopulation():
    population = []  
    bits = [0 , 1]
    for i in range(8):
        chromosome = "" 
        for j in range(11) :
            chromosome += str(bits[random.randint(0, 1)])
        population.append(chromosome)
    return population


In [ ]:
def convertBinaryToDecimal(chromosome):
    decimal = 0
    for i in range(len(chromosome)):
        decimal += int(chromosome[i]) * (2 ** (len(chromosome) - 1 - i))
    return decimal
   

In [10]:
import random   
firstPopulation = createPopulation()
print(firstPopulation)


['01011010011', '10000001010', '01101111000', '11101010110', '11011100100', '01001100110', '00111101100', '00011001110']


In [11]:
print(convertBinaryToDecimal(firstPopulation[0]))

723
